In [1]:
!jt -t chesterish

In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import pprint
import itertools
import numpy as np

pd.set_option('display.max_columns', None)


In [2]:
#define as static variables for now, must be updated via form info
api_key = ''
watcher = LolWatcher(api_key)
region = 'na1'
gamemode = 'ARAM' #CLASSIC = ranked? ; NEXUSBLITZ = nexus blitz ; ARAM = ARAM
name = 'Sir Thiccboi'
champion_id = 81

In [26]:
class user_info:
    """
    Returns df of user info from a given match (so far). 
    """
    
    #define private variables to use in class
    user = watcher.summoner.by_name(region, name)

    def __init__(self, api_key, region, name, champion_id, gamemode):
        self.api_key = api_key
        self.region = region
        self.champion_id = champion_id
        self.gamemode = gamemode
        
    def rank_stats(self): 
        #league, division, games played, etc.
        encrypted_summoner_id = user['id']
        self.rank_stats = watcher.league.by_summoner(region, self.user['id'])
        return self.rank_stats
    
    def match_data(self): 
        self.matches = watcher.match.matchlist_by_account(region, self.user['accountId'])    

        self.last_match = self.matches['matches'][0]
    
       # self.last_match = self.matches['matches'][0]
        self.last_match_data = watcher.match.by_id(region, self.last_match['gameId'])
        
        m = self.last_match_data
        #pprint.pprint(m)
        
        #n is for each "participant" or player in the match
        def gd():
            n = [] #dump raw stats into here from participants
            for row in m['participants']:            
                m_row = {}
                m_row['champion'] = row['championId']
                m_row['spell1'] = row['spell1Id']
                m_row['spell2'] = row['spell2Id']
                m_row['teamId'] = row['teamId']
                m_row['win'] = row['stats']['win']
                m_row['kills'] = row['stats']['kills']
                m_row['deaths'] = row['stats']['deaths']
                m_row['assists'] = row['stats']['assists']
                m_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
                m_row['totalDamageDealtToChampions'] = row['stats']['totalDamageDealtToChampions']
                m_row['goldEarned'] = row['stats']['goldEarned']
                m_row['champLevel'] = row['stats']['champLevel']
                m_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
                m_row['item0'] = row['stats']['item0']
                m_row['item1'] = row['stats']['item1']
                m_row['item2'] = row['stats']['item2']
                m_row['item3'] = row['stats']['item3']
                m_row['item4'] = row['stats']['item4']
                m_row['item5'] = row['stats']['item5']
                m_row['item6'] = row['stats']['item6']
                m_row['visionWardsBoughtInGame'] = row['stats']['visionWardsBoughtInGame']
                m_row['visionScore'] = row['stats']['visionScore']
                m_row['creepsPerMinDeltas'] = row['timeline']['creepsPerMinDeltas']
                #m_row['csDiffPerMinDeltas'] = row['timeline']['csDiffPerMinDeltas']
                m_row['goldPerMinDeltas'] = row['timeline']['goldPerMinDeltas']
                m_row['lane'] = row['timeline']['lane']
                m_row['ccScore'] = row['stats']['totalTimeCrowdControlDealt']
                m_row['perkPrimaryStyle'] = row['stats']['perkPrimaryStyle']
                m_row['perkSubStyle'] = row['stats']['perkSubStyle']
                n.append(m_row)    
            return n
        
        n = gd()
        for i in range(0,len(n)):
            n[i]['summonerName'] = m['participantIdentities'][i]['player']['summonerName']
            n[i]['profileIcon'] = m['participantIdentities'][i]['player']['profileIcon']
        
        latest = watcher.data_dragon.versions_for_region(region)['n']['champion']
        static_champ_list = watcher.data_dragon.champions(latest, False, 'en_US') 
        
        # champ static list data to dict for looking up
        def g_c(n):
            
            champ_dict = {}
            for key in static_champ_list['data']:
                row = static_champ_list['data'][key]
                champ_dict[row['key']] = row['id']
            for row in n:
                #print(str(row['champion']) + ' ' + champ_dict[str(row['champion'])])
                row['championName'] = champ_dict[str(row['champion'])]
            
            df = pd.DataFrame(n)
            return df
        
        df = g_c(n)
        
        #add in extra columns
        df['gameDuration'] = m['gameDuration'] / 60
        df['gameMode'] = m['gameMode']
        df['kda'] = ((df['kills'] + df['assists']) / df['deaths']).round(2)
        df['killParticipation'] = ((df['kills'] + df['assists'])/ df.groupby('teamId')['kills'].transform(np.sum) * 100).astype(int)        
        df['minionsKilledPerMinute'] = (df['totalMinionsKilled'] / df['gameDuration']).round(1) 

        pprint.pprint(m['teams'])
        
        def g_t(df, m): 
            teamId = set(df['teamId']) 
            print(teamId)
            for i in range(0, len(teamId)): 
                check = (i+1)*100 #change i=0,1 to teamId 100, 200 respectively
                print(check)
#                 for j in range(0, len(df['teamId'].values)): #loop thru team & check value 
                if str(check) == str(df['teamId'][i]): 
                    print(check, df['teamId'])
                    df['firstBlood'] = m['teams'][i]['firstBlood']
                    df['baronKills'] = m['teams'][i]['baronKills']
                    df['firstTower'] = m['teams'][i]['firstTower']
                    df['firstRiftHerals'] = m['teams'][i]['firstRiftHerald']
                    df['towerKills'] = m['teams'][i]['towerKills']
                    df['inhibitorKills'] = m['teams'][i]['inhibitorKills']
                    df['dragonKills'] = m['teams'][i]['dragonKills']
                    df['riftHeraldKills'] = m['teams'][i]['riftHeraldKills']
            return df

        df = g_t(df, m)
        
        return df, m

user_1 = user_info(api_key, name, region, champion_id, gamemode)

In [27]:
df, m = user_1.match_data()

[{'bans': [{'championId': 2, 'pickTurn': 1},
           {'championId': 39, 'pickTurn': 2},
           {'championId': 360, 'pickTurn': 3},
           {'championId': -1, 'pickTurn': 4},
           {'championId': 161, 'pickTurn': 5}],
  'baronKills': 0,
  'dominionVictoryScore': 0,
  'dragonKills': 1,
  'firstBaron': False,
  'firstBlood': False,
  'firstDragon': False,
  'firstInhibitor': False,
  'firstRiftHerald': False,
  'firstTower': False,
  'inhibitorKills': 0,
  'riftHeraldKills': 0,
  'teamId': 100,
  'towerKills': 2,
  'vilemawKills': 0,
  'win': 'Fail'},
 {'bans': [{'championId': 106, 'pickTurn': 6},
           {'championId': 105, 'pickTurn': 7},
           {'championId': 23, 'pickTurn': 8},
           {'championId': 141, 'pickTurn': 9},
           {'championId': 147, 'pickTurn': 10}],
  'baronKills': 1,
  'dominionVictoryScore': 0,
  'dragonKills': 4,
  'firstBaron': True,
  'firstBlood': True,
  'firstDragon': True,
  'firstInhibitor': True,
  'firstRiftHerald': True,
  'fir

In [28]:
df

,champion,spell1,spell2,teamId,win,kills,deaths,assists,totalDamageDealt,totalDamageDealtToChampions,goldEarned,champLevel,totalMinionsKilled,item0,item1,item2,item3,item4,item5,item6,visionWardsBoughtInGame,visionScore,creepsPerMinDeltas,goldPerMinDeltas,lane,ccScore,perkPrimaryStyle,perkSubStyle,summonerName,profileIcon,championName,gameDuration,gameMode,kda,killParticipation,minionsKilledPerMinute,firstBlood,baronKills,firstTower,firstRiftHerals,towerKills,inhibitorKills,dragonKills,riftHeraldKilsl
0,236,4,7,100,False,3,11,8,102813,14644,10190,15,159,1055,3091,6673,3111,3133,1036,3340,1,29,"{'10-20': 4.3, '0-10': 6.800000000000001, '20-...","{'10-20': 288.7, '0-10': 252.7, '20-30': 368}",BOTTOM,0,8000,8100,Sir Diemos,4570,Lucian,33.083333,CLASSIC,1.00,36,4.8,False,0,False,False,2,0,1,0
1,7,4,14,100,False,8,12,6,91127,25224,10120,14,103,6655,3165,1058,1058,0,3020,3340,2,22,"{'10-20': 3.5, '0-10': 3.7, '20-30': 3.1}","{'10-20': 316.7, '0-10': 237.9, '20-30': 357.3}",MIDDLE,32,8100,8200,XxPapaTxX,4403,Leblanc,33.083333,CLASSIC,1.17,46,3.1,False,0,False,False,2,0,1,0
2,31,4,12,100,False,10,5,6,148513,30991,13716,17,188,3193,3047,6662,3075,4401,0,3340,1,20,"{'10-20': 7.4, '0-10': 6.3, '20-30': 4.5}","{'10-20': 410.2, '0-10': 390.9, '20-30': 460.4}",TOP,1463,8400,8200,Thild VerRequis,3496,Chogath,33.083333,CLASSIC,3.20,53,5.7,False,0,False,False,2,0,1,0
3,26,3,4,100,False,3,8,7,98069,23101,9593,14,77,3853,6655,3041,3020,1052,1058,3340,0,28,"{'10-20': 3.2, '0-10': 0.5, '20-30': 2.8}","{'10-20': 307, '0-10': 169.39999999999998, '20...",MIDDLE,271,8200,8100,azoom,536,Zilean,33.083333,CLASSIC,1.25,33,2.3,False,0,False,False,2,0,1,0
4,62,4,11,100,False,6,10,12,114630,21493,11564,15,25,6630,3156,3111,3123,3077,0,3364,9,38,"{'10-20': 0.7, '0-10': 0.8, '20-30': 1}","{'10-20': 359.2, '0-10': 291.7, '20-30': 324.4}",JUNGLE,184,8000,8100,Sir Thiccboi,4855,MonkeyKing,33.083333,CLASSIC,1.80,60,0.8,False,0,False,False,2,0,1,0
5,34,4,14,200,True,10,4,10,155175,31974,13722,17,152,3157,6655,3020,1056,3135,1058,3340,2,37,"{'10-20': 5.4, '0-10': 4.6, '20-30': 4.4}","{'10-20': 411.9, '0-10': 239.3, '20-30': 514}",MIDDLE,1008,8100,8200,DoedDoeBird,3614,Anivia,33.083333,CLASSIC,5.00,43,4.6,False,0,False,False,2,0,1,0
6,234,4,11,200,True,9,10,10,237442,31271,14807,16,69,6672,3153,3047,3053,3091,0,3364,6,35,"{'10-20': 1.7000000000000002, '0-10': 1, '20-3...","{'10-20': 396.4, '0-10': 326.5, '20-30': 522.8}",JUNGLE,259,8000,8400,boysbathroom,953,Viego,33.083333,CLASSIC,1.90,41,2.1,False,0,False,False,2,0,1,0
7,420,4,12,200,True,7,7,9,174060,28366,12795,17,178,3065,6630,3111,3053,0,0,3340,1,39,"{'10-20': 6.699999999999999, '0-10': 5.8, '20-...","{'10-20': 326.70000000000005, '0-10': 268.9, '...",TOP,30,8000,8100,SCOUT LOL,4779,Illaoi,33.083333,CLASSIC,2.29,34,5.4,False,0,False,False,2,0,1,0
8,29,14,4,200,True,18,6,8,189389,53218,16038,16,161,3089,4636,3041,3115,3006,3057,3340,1,24,"{'10-20': 7.199999999999999, '0-10': 5.8, '20-...","{'10-20': 549, '0-10': 266.9, '20-30': 568}",BOTTOM,588,8100,8000,Dozax,3367,Twitch,33.083333,CLASSIC,4.33,56,4.9,False,0,False,False,2,0,1,0
9,117,4,3,200,True,2,3,27,33087,8694,10589,15,44,3860,3158,3504,6617,6616,2055,3364,2,45,"{'10-20': 1.4, '0-10': 1.2999999999999998, '20...","{'10-20': 316.5, '0-10': 164.4, '20-30': 372.2}",BOTTOM,261,8200,8300,DaddyOompaLoompa,3159,Lulu,33.083333,CLASSIC,9.67,63,1.3,False,0,False,False,2,0,1,0


In [ ]:
m['teams']

In [ ]:
df = df.reindex(sorted(df.columns), axis=1)
df